In [1]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label
import math

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [2]:
num_qubits = 2
max_weight = 2
term_dict = {("H", "XI"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [3]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    union = used_indices_1.union(used_indices_2)
    if len(union) > 0 and len(union) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
sign_iterator = list(product([1,-1], repeat=num_qubits))
awk_iterator = list(i[0] for i in product(sign_iterator, [p for p in measure_string_iterator]))
prep_string_iterator = product([math.prod([item for item in sign_tuple]) for sign_tuple in sign_iterator],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))

In [4]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)

In [5]:
hs_experiment

[('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XY")),
  stim.PauliString("+XX")),
 ('h',
  s

In [6]:
import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[:] for s in el[-5:-1])
            sign_string = el[-4]
            egen = ",".join(str(s) for s in el[:-5])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])

In [7]:
jacobian_coef_dict["columns"]
jacobian_coef_dict["index"]

OrderedSet(['+XX,(1, 1),+XX,+X_', '+XX,(1, 1),+XX,+_X', '+XX,(1, 1),+XX,+XX', '+XX,(1, 1),+XY,+X_', '+XX,(1, 1),+XY,+_Y', '+XX,(1, 1),+XY,+XY', '+XX,(1, 1),+XZ,+X_', '+XX,(1, 1),+XZ,+_Z', '+XX,(1, 1),+XZ,+XZ', '+XX,(1, 1),+YX,+Y_', '+XX,(1, 1),+YX,+_X', '+XX,(1, 1),+YX,+YX', '+XX,(1, 1),+YY,+Y_', '+XX,(1, 1),+YY,+_Y', '+XX,(1, 1),+YY,+YY', '+XX,(1, 1),+YZ,+Y_', '+XX,(1, 1),+YZ,+_Z', '+XX,(1, 1),+YZ,+YZ', '+XX,(1, 1),+ZX,+Z_', '+XX,(1, 1),+ZX,+_X', '+XX,(1, 1),+ZX,+ZX', '+XX,(1, 1),+ZY,+Z_', '+XX,(1, 1),+ZY,+_Y', '+XX,(1, 1),+ZY,+ZY', '+XX,(1, 1),+ZZ,+Z_', '+XX,(1, 1),+ZZ,+_Z', '+XX,(1, 1),+ZZ,+ZZ', '+XY,(1, 1),+XX,+X_', '+XY,(1, 1),+XX,+_X', '+XY,(1, 1),+XX,+XX', '+XY,(1, 1),+XY,+X_', '+XY,(1, 1),+XY,+_Y', '+XY,(1, 1),+XY,+XY', '+XY,(1, 1),+XZ,+X_', '+XY,(1, 1),+XZ,+_Z', '+XY,(1, 1),+XZ,+XZ', '+XY,(1, 1),+YX,+Y_', '+XY,(1, 1),+YX,+_X', '+XY,(1, 1),+YX,+YX', '+XY,(1, 1),+YY,+Y_', '+XY,(1, 1),+YY,+_Y', '+XY,(1, 1),+YY,+YY', '+XY,(1, 1),+YZ,+Y_', '+XY,(1, 1),+YZ,+_Z', '+XY,(1, 1),+YZ,+YZ'

PREP STATES ARE NOT PROPER SUMS YET SO FIX THAT

In [8]:
df

,"H,+X_","H,+Y_","H,+Z_","H,+_X","H,+_Y","H,+_Z","H,+XX","H,+XY","H,+XZ","H,+YX",...,"A,+YY,+YZ","A,+YY,+ZX","A,+YY,+ZY","A,+YY,+ZZ","A,+YZ,+ZX","A,+YZ,+ZY","A,+YZ,+ZZ","A,+ZX,+ZY","A,+ZX,+ZZ","A,+ZY,+ZZ"
"+XX,(1, 1),+XX,+X_",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"+XX,(1, 1),+XX,+_X",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"+XX,(1, 1),+XX,+XX",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"+XX,(1, 1),+XY,+X_",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"+XX,(1, 1),+XY,+_Y",0,0,0,0,0,2,0,0,2,0,...,0,2,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"+ZZ,(-1, -1),+ZY,+_Y",0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
"+ZZ,(-1, -1),+ZY,+ZY",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-4,0
"+ZZ,(-1, -1),+ZZ,+Z_",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"+ZZ,(-1, -1),+ZZ,+_Z",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-4,0,0


In [9]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [10]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in range(num_qubits)))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1, idle_string=global_idle_string
    )
idle_experiments

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

[Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gy:0Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gy:0Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gx:0Gy:1]Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gx:0Gy:1]Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gx:0Gx:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gx:0Gx:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gx:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gx:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gy:1Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gy:1Gy:1Gy:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gx:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1Gx:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1@(0,1)),
 Circuit([Gy:0Gx:1]Gx:1Gx:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gx:1]Gx:1Gx:1Gi:0:1Gi:0:1[Gy:0Gy

In [11]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 10000000, seed=8675309, sample_error="none"
)

In [12]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    ic(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        ic(pauli_fidpair)
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            infos_for_this_fidpair[out] = info
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [13]:
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

In [14]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [15]:
df.index

Index(['+XX,(1, 1),+XX,+X_', '+XX,(1, 1),+XX,+_X', '+XX,(1, 1),+XX,+XX',
       '+XX,(1, 1),+XY,+X_', '+XX,(1, 1),+XY,+_Y', '+XX,(1, 1),+XY,+XY',
       '+XX,(1, 1),+XZ,+X_', '+XX,(1, 1),+XZ,+_Z', '+XX,(1, 1),+XZ,+XZ',
       '+XX,(1, 1),+YX,+Y_',
       ...
       '+ZZ,(-1, -1),+YZ,+YZ', '+ZZ,(-1, -1),+ZX,+Z_', '+ZZ,(-1, -1),+ZX,+_X',
       '+ZZ,(-1, -1),+ZX,+ZX', '+ZZ,(-1, -1),+ZY,+Z_', '+ZZ,(-1, -1),+ZY,+_Y',
       '+ZZ,(-1, -1),+ZY,+ZY', '+ZZ,(-1, -1),+ZZ,+Z_', '+ZZ,(-1, -1),+ZZ,+_Z',
       '+ZZ,(-1, -1),+ZZ,+ZZ'],
      dtype='object', length=972)

In [16]:
obs_error_rates_by_exp

{('XX', 'XX', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XX', ' _X'): -1.6953455232808294e-16,
 ('XX', 'XX', ' XX'): -1.6953455232808294e-16,
 ('XX', 'XY', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XY', ' _Y'): 0.0,
 ('XX', 'XY', ' XY'): 0.0,
 ('XX', 'XZ', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XZ', ' _Z'): 0.0,
 ('XX', 'XZ', ' XZ'): 0.0,
 ('XX', 'YX', ' Y_'): 0.0,
 ('XX', 'YX', ' _X'): -1.6953455232808294e-16,
 ('XX', 'YX', ' YX'): 0.0,
 ('XX', 'YY', ' Y_'): 0.0,
 ('XX', 'YY', ' _Y'): 0.0,
 ('XX', 'YY', ' YY'): 0.0,
 ('XX', 'YZ', ' Y_'): 0.0,
 ('XX', 'YZ', ' _Z'): 0.0,
 ('XX', 'YZ', ' YZ'): 0.0,
 ('XX', 'ZX', ' Z_'): 0.0,
 ('XX', 'ZX', ' _X'): -1.6953455232808294e-16,
 ('XX', 'ZX', ' ZX'): 0.0,
 ('XX', 'ZY', ' Z_'): 0.0,
 ('XX', 'ZY', ' _Y'): 0.0,
 ('XX', 'ZY', ' ZY'): 0.0,
 ('XX', 'ZZ', ' Z_'): 0.0,
 ('XX', 'ZZ', ' _Z'): 0.0,
 ('XX', 'ZZ', ' ZZ'): 0.0,
 ('XY', 'XX', ' X_'): -1.6953455232808294e-16,
 ('XY', 'XX', ' _X'): 0.0,
 ('XY', 'XX', ' XX'): 0.0,
 ('XY', 'XY', ' X_'): -4.29825

In [17]:
j = df.to_numpy()

In [18]:
np.linalg.matrix_rank(j)

240

In [19]:
jinv = np.linalg.pinv(j)

In [20]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [21]:
dict(zip(df.columns,intrins_errs))

{'H,+X_': -1.6940658945086007e-21,
 'H,+Y_': -4.91057551693686e-21,
 'H,+Z_': -9.581721055614936e-21,
 'H,+_X': 2.371692252312041e-20,
 'H,+_Y': -5.460268760695631e-22,
 'H,+_Z': -4.748150406287276e-20,
 'H,+XX': 9.999999999999996e-05,
 'H,+XY': -4.6501545662459695e-20,
 'H,+XZ': 6.604287578513092e-20,
 'H,+YX': 1.1278656774074628e-20,
 'H,+YY': 7.057549332999739e-20,
 'H,+YZ': 1.859241903357765e-20,
 'H,+ZX': -3.7705294649264046e-20,
 'H,+ZY': -1.835922412086941e-20,
 'H,+ZZ': -2.117582368135751e-21,
 'S,+X_': 9.848198490801985e-18,
 'S,+Y_': 8.20911405859927e-19,
 'S,+Z_': 7.670013628477324e-19,
 'S,+_X': 9.816167654108181e-18,
 'S,+_Y': 8.118850990988515e-19,
 'S,+_Z': 7.524731313736399e-19,
 'S,+XX': 2.999999885288232e-08,
 'S,+XY': 7.338550262236948e-19,
 'S,+XZ': 8.357175706205391e-19,
 'S,+YX': 7.871584987158194e-19,
 'S,+YY': 1.9128035154735407e-18,
 'S,+YZ': 2.0346189133189945e-18,
 'S,+ZX': 6.884616572802309e-19,
 'S,+ZY': 1.9522497167593704e-18,
 'S,+ZZ': 1.8985110188808885e